<a href="https://colab.research.google.com/github/samuelagyemang2012/Sentiment-Analysis/blob/master/Sentence_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install sentence-splitter

In [3]:
pip install langdetect

In [4]:
pip install clean-text

In [5]:
pip install unidecode

In [29]:
import spacy
import re
import csv
import string
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("stopwords")
import numpy as np
from sentence_splitter import SentenceSplitter, split_text_into_sentences
from langdetect import detect
import numpy as np
from cleantext import clean
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Import reviews
reviews_data_path = "/content/drive/MyDrive/ML Data/hotel_reviews.csv"

In [8]:
# Fetch reviews
df = pd.read_csv(reviews_data_path)

print(df.head())
print("")


                  address categories  ... reviews.username reviews.userProvince
0  Riviera San Nicol 11/a     Hotels  ...      Russ (kent)                  NaN
1  Riviera San Nicol 11/a     Hotels  ...       A Traveler                  NaN
2  Riviera San Nicol 11/a     Hotels  ...             Maud                  NaN
3  Riviera San Nicol 11/a     Hotels  ...            Julie                  NaN
4  Riviera San Nicol 11/a     Hotels  ...         sungchul                  NaN

[5 rows x 19 columns]



In [9]:
# Print columns in df
for col in df.columns:
    print(col)

address
categories
city
country
latitude
longitude
name
postalCode
province
reviews.date
reviews.dateAdded
reviews.doRecommend
reviews.id
reviews.rating
reviews.text
reviews.title
reviews.userCity
reviews.username
reviews.userProvince


In [10]:
# Fetch hotels and reviews
reviews_subset = df[["name", "reviews.text"]]
hotel_names = reviews_subset.name.unique()
hotel_names = hotel_names.tolist()
hotel_names.sort()

for h in hotel_names:
  print(h)

1785 Inn
1900 House
40 Berkeley Hostel
A Bed & Breakfast In Cambridge
A Swallow's Nest Motel
Abbeville Inn
Acorn Motor Inn
Adobe Nido Bed & Breakfast
Adria Motor Inn
Advance Motel
Agate Beach Motel
Albert's Motel
Alpine Motel
Alyssa's Motel
Ambassador Inn Albuquerque
Ambassadors Inn and Suites
Ambiance
America's Best Value Inn
American Star Inn and Suites Atlantic City
Americana Resort Properties
Americas Best Value Inn
Americas Best Value Inn - Medical Center/airport
Americas Best Value Inn and Suites
Americinn Coralville
Americinn Lodge & Suites Appleton
Americinn Lodge Suites Austin
Americinn Lodge Suites Oscoda - Ausable River
Americinn Lodge Suites Princeton
Americinn Lodge and Suites
Amerihost Inn
Amisos Hotel
Anaheim Marriott Suites
Anglers Inn
Antlers Inn
Arcola Inn
Arion
Arizona Inn of Prescott Valley
Arrowhead Motel and Rv Park
Aspen Alps
Aspen Haus Bed and Breakfast
Astro Motel
Atlanta Marriott Northwest At Galleria
Atlantic House
Avalon Hotel Group
Azul Beach Hotel By Karis

In [11]:
# Get hotel review count
hotel_review_num = []
for h in hotel_names:
  num_reviews = len((df[(df['name'] == h)]))
  hotel_review_num.append([h,num_reviews])

print(hotel_review_num[76])

['Best Western Plus French Quarter Landmark Hotel', 215]


In [12]:
hotel = "Hotel Russo Palace"

hotel_df = df.loc[df['name'] == hotel]
hotel_reviews = hotel_df["reviews.text"].tolist()

In [13]:
len(hotel_reviews)

57

In [14]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [15]:
# Load Spacy model and increase max numer if chars to be processed
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 2000000

In [23]:
# Fetch stopwords
en_stopwords = set(nltk.corpus.stopwords.words('english'))
 
# Prepocessing for sentences alg1
def preprocess(sentences):
  data=[]
  for x in sentences:
    try:
      if detect(x) == 'en':
        x = x.lower()
        x = ' '.join([word for word in x.split(' ') if word not in en_stopwords])
        x = x.encode('ascii', 'ignore').decode()
        x = re.sub(r'https*\S+', ' ', x)
        x = re.sub(r'@\S+', ' ', x)
        x = re.sub(r'#\S+', ' ', x)
        x = re.sub(r'\'\w+', '', x)
        x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
        x = re.sub(r'\w*\d+\w*', '', x)
        x = re.sub(r'\s{2,}', ' ', x)
 
        data.append(x)
    except:
      pass
  return data

# Prepocessing for sentences alg2, uses clean text package
def clean_text(sentences):
  data=[]
  for x in sentences:
    try: 
      if detect(x) == 'en':
        x = x.lower()
        x = ' '.join([word for word in x.split(' ') if word not in en_stopwords])
        x = clean(x,
          fix_unicode=True,               # fix various unicode errors
          to_ascii=True,                  # transliterate to closest ASCII representation
          lower=True,                     # lowercase text
          no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
          no_urls=True,                  # replace all URLs with a special token
          no_emails=True,                # replace all email addresses with a special token
          no_phone_numbers=True,         # replace all phone numbers with a special token
          no_numbers=True,               # replace all numbers with a special token
          no_digits=True,                # replace all digits with a special token
          no_currency_symbols=True,      # replace all currency symbols with a special token
          no_punct=True,                 
          replace_with_punct="",          
          replace_with_url="<URL>",
          replace_with_email="<EMAIL>",
          replace_with_phone_number="<PHONE>",
          replace_with_number="<NUMBER>",
          replace_with_digit="0",
          replace_with_currency_symbol="<CUR>",
          lang="en")                                
        data.append(x)
    except Exception as e:
      print(e)
      pass

  return data

# Split a document into sentences
def split_sentences(text, lang):
  splitter = SentenceSplitter(language=lang)
  sents = splitter.split(text=text)
  return sents

# Split a list of document into sentences
def split_sentence_in_doc(doc,lang):
  sentences = []
  for d in hotel_reviews:
    sents = split_text_into_sentences(d, lang)
    for s in sents:
      try:
        if detect(s) == lang:
          sentences.append(s)
      except:
        pass
  
  return sentences

# Takes a sentence, tokenizes it, gets the word embeddings of every token and calculates the centroids of all tokens
def embed_tokens(tokens, nlp):
    """Return the centroid of the embeddings for the given tokens.
    Out-of-vocabulary tokens are cast aside. Stop words are also
    discarded. An array of 0s is returned if none of the tokens
    are valid.
    """
    for token in tokens:
      lexemes = (nlp.vocab[token] for token in tokens)

      vectors = np.asarray([
        lexeme.vector
        for lexeme in lexemes
        if lexeme.has_vector and not lexeme.is_stop and len(lexeme.text) > 1
      ])

      if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
      else:
        width = nlp.meta['vectors']['width']  # typically 300
        centroid = np.zeros(width)

    return centroid

# Takes a document and calculates the centroids for each sentence
# https://www.wikiwand.com/en/Centroid
def get_centroids(sentences, nlp):
  centroids = []
  for s in sentences:
    s = s.lower()
    s = s.split(' ')
    cen = embed_tokens(s, nlp)
    centroids.append(cen)
  
  return centroids

# COnvert labels to vectors
def vectorize_labels(labels):
  label_vectors = np.asarray([
    embed_tokens(label.split(' '), nlp)
    for label in labels
  ])

  return label_vectors

# Takes a list of sentence centroids and tags them according to a list of labels using an ml model. Currently, K-Nearest Neighbour
def tag_sentences(centroids, sentences, labels, model):
  tags=[]
  for i in range(len(centroids)):
    closest_label = model.kneighbors([centroids[i]], return_distance=False)[0, 0]
    tags.append(labels[closest_label])
    
  return tags

# Takes a list of sentence centroids and tags them according to a list of labels using the maximum cosine similarity for each label
def tag_sentences2(centroids, sentences, label_vectors, labels):
  cosims=[]
  tags=[]
  for i in range(len(centroids)):
    for j in range(len(label_vectors)):
      v1 = centroids[i].reshape(1, -1)
      v2 = label_vectors[j].reshape(1, -1)
      cosim = cosine_similarity(v1, v2)
      cosims.append(cosim[0][0])
    # print(sentences[i])
    # print(labels[cosims.index(max(cosims))])
    tags.append(labels[cosims.index(max(cosims))])
    # print(i)
    # print("")
    cosims.clear()

  return tags

In [26]:
# Example of sentence splitting
example1=hotel_reviews[1]
sents1 = split_sentences(example1,'en')
for s in sents1:
  print(s)

Really lovely hotel.
Stayed on the very top floor and were surprised by a Jacuzzi bath we didn't know we were getting!
Staff were friendly and helpful and the included breakfast was great!
Great location and great value for money.
Didn't want to leave!


In [27]:
# Split sentences in reviews
sentences = split_sentence_in_doc(hotel_reviews,'en')
print(len(sentences))
for s in sentences[0:5]:
  print(s)

132
Pleasant 10 min walk along the sea front to the Water Bus. restaurants etc. Hotel was comfortable breakfast was good - quite a variety.
Room aircon didn't work very well.
Really lovely hotel.
Stayed on the very top floor and were surprised by a Jacuzzi bath we didn't know we were getting!
Staff were friendly and helpful and the included breakfast was great!


In [30]:
# Improve alg by NER tagging for COUNTRIES, CITIES, etc. and replacing with generic words such as location, place
# Improve alg by correction all misplelt words using enchant library or Peter Norvig’s spelling corrector
# Try differnt ml algs for calssification and evaluate them
# https://maxhalford.github.io/blog/unsupervised-text-classification/

labels = ['room','breakfast','staff','location']

centroids = get_centroids(sentences, nlp)
label_vectors = vectorize_labels(labels)

# Init model- Neighrest Neighbour
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(label_vectors)

tags_alg1 = tag_sentences(centroids, sentences, labels, neigh)

In [31]:
tags_alg2 = tag_sentences2(centroids, sentences, label_vectors, labels)

In [32]:
fields = ['Sentences', 'Tags_1', 'Tags_2']
sents_tags = []

for i in range(len(sentences)):
  s = sentences[i]
  t1 = tags_alg1[i]
  t2 = tags_alg2[i]
  sents_tags.append([s, t1, t2])
    
# data rows of csv file 
  
with open(hotel+".csv", 'w') as f: 
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(sents_tags)

In [ ]:
# print(label_vectors[0].shape)
# tag_sentences2(centroids[0], label_vectors,labels)
# print(sentences[0])
# cosims=[]
# for i in range(len(label_vectors)):
#   v1 = centroids[0].reshape(1, -1)
#   v2 = label_vectors[i].reshape(1, -1)
#   cosim = cosine_similarity(v1, v2)
#   cosims.append(cosim[0][0])
# print(labels[cosims.index(max(cosims))])
# cosims.clear()
# print(v1.shape)
# print(v2.shape)
# cosim = cosine_similarity(v1, v2)
# print(cosim)